In [1]:
import enum
import glob
import os
from hashlib import new
from pathlib import Path
import time
from itertools import product

import functools

import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from snorkel.labeling.model import LabelModel as LMsnorkel
from snorkel.labeling.model import MajorityLabelVoter

from sklearn.model_selection import train_test_split
import itertools
import ast

In [2]:
# Initialize seed
seed_i = 0

In [3]:
import joblib
import json
import collections

In [4]:
from sklearn.exceptions import UndefinedMetricWarning

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [5]:
def list2Nested(l, nested_length):
    return [l[i:i+nested_length] for i in range(0, len(l), nested_length)]

In [6]:
labelModel_mapper_LF = {1:1, -1:0, 0:-1}

In [7]:
import LMutils

train_file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v4/gt/train_ebm_labels_tui_pio3.tsv'
training_data = pd.read_csv(train_file, sep='\t', header=0)

ebm_test_file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v4/gt/test_ebm_labels_tui_pio3.tsv'
test_ebm_data = pd.read_csv(ebm_test_file, sep='\t', header=0)
test_ebm_data.rename( columns={'Unnamed: 0':'series'}, inplace=True )

physio_test_file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v4/gt/test_physio_labels_tui_pio3.tsv'
test_physio_data = pd.read_csv(physio_test_file, sep='\t', header=0)
test_physio_data.rename( columns={'Unnamed: 0':'series'}, inplace=True )

ebm_test_corrected_file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v4/gt/test_ebm_correctedlabels_tui_pio3.tsv'
test_ebm_corrected_data = pd.read_csv(ebm_test_corrected_file, sep='\t', header=0)
test_ebm_corrected_data.rename( columns={'Unnamed: 0':'series'}, inplace=True )

In [8]:
def flatten_df(df):

    df_series = [ index for index, value in df.tokens.items() for word in ast.literal_eval(value) ]
    df_tokens = [ word for index, value in df.tokens.items() for word in ast.literal_eval(value) ]
    df_pos = [ word for index, value in df.pos.items() for word in ast.literal_eval(value) ]
    df_offsets = [ word for index, value in df.offsets.items() for word in ast.literal_eval(value) ]


    df_p = [ int(lab) for index, value in df.p.items() for lab in ast.literal_eval(value) ]
    df_p_fine = [ int(lab) for index, value in df.p_f.items() for lab in ast.literal_eval(value) ]
    df_i = [ int(lab) for index, value in df.i.items() for lab in ast.literal_eval(value) ]
    df_i_fine = [ int(lab) for index, value in df.i_f.items() for lab in ast.literal_eval(value) ]
    df_o = [ int(lab) for index, value in df.o.items() for lab in ast.literal_eval(value) ]
    df_o_fine = [ int(lab) for index, value in df.o_f.items() for lab in ast.literal_eval(value) ]
    df_s = [ int(lab) for index, value in df.s.items() for lab in ast.literal_eval(value) ]
    df_s_fine = [ int(lab) for index, value in df.s_f.items() for lab in ast.literal_eval(value) ]
    
    df_flattened = pd.DataFrame({ 'series': df_series,
                        'tokens' : df_tokens,
                        'offsets': df_offsets,
                        'pos': df_pos,
                        'p' : df_p,
                        'i' : df_i,
                        'o' : df_o,
                        's' : df_s,
                        'p_f' : df_p_fine,
                        'i_f' : df_i_fine,
                        'o_f' : df_o_fine,
                        's_f' : df_s_fine})
    
    return df_flattened

In [9]:
# Flatten the dataframes (currently only the training dataframe and test ebm dataframe with corrected labels can be flattened)
data_df = flatten_df(training_data)
test_ebm_data = flatten_df(test_ebm_data)
test_ebm_corr_df = flatten_df(test_ebm_corrected_data)

In [10]:
series = [
    data_df.series.to_numpy() ,
    test_ebm_data.series.to_numpy() ,
    test_physio_data.series.to_numpy(),   
    test_ebm_corr_df.series.to_numpy()
]


sents = [
    data_df.tokens.to_numpy() ,
    test_ebm_data.tokens.to_numpy() ,
    test_physio_data.tokens.to_numpy(),   
    test_ebm_corr_df.tokens.to_numpy()    
]


part_of_speech = [
    data_df.pos.to_numpy() ,
    test_ebm_data.pos.to_numpy() ,
    test_physio_data.pos.to_numpy(),   
    test_ebm_corr_df.pos.to_numpy()     
]


offsets = [
    data_df.offsets.to_numpy() ,
    test_ebm_data.offsets.to_numpy() ,
    test_physio_data.offsets.to_numpy(),   
    test_ebm_corr_df.offsets.to_numpy() 
]


Y_p = [
    data_df.p.to_numpy() , # 0 -7
    data_df.p_f.to_numpy() , # 1 -6
    test_ebm_data.p.to_numpy() , # 2 -5
    test_ebm_data.p_f.to_numpy() , # 3 -4
    test_physio_data.p.to_numpy(),  # 4 -3
    test_ebm_corr_df.p.to_numpy(),   # 5 -2
    test_ebm_corr_df.p_f.to_numpy() # 6 -1
]


Y_i = [
    data_df.i.to_numpy() , # 0 -7
    data_df.i_f.to_numpy() , # 1 -6
    test_ebm_data.i.to_numpy() , # 2 -5
    test_ebm_data.i_f.to_numpy() , # 3 -4
    test_physio_data.i.to_numpy(),  # 4 -3
    test_ebm_corr_df.i.to_numpy(),   # 5 -2
    test_ebm_corr_df.i_f.to_numpy() # 6 -1
]


Y_o = [
    data_df.o.to_numpy() , # 0 -7
    data_df.o_f.to_numpy() , # 1 -6
    test_ebm_data.o.to_numpy() , # 2 -5
    test_ebm_data.o_f.to_numpy() , # 3 -4
    test_physio_data.o.to_numpy(),  # 4 -3
    test_ebm_corr_df.o.to_numpy(),   # 5 -2
    test_ebm_corr_df.o_f.to_numpy() # 6 -1
]

Y_s = [
    data_df.s.to_numpy() , # 0 -7
    data_df.s_f.to_numpy() , # 1 -6
    test_ebm_data.s.to_numpy() , # 2 -5
    test_ebm_data.s_f.to_numpy() , # 3 -4
    test_physio_data.s.to_numpy(),  # 4 -3
    test_ebm_corr_df.s.to_numpy(),   # 5 -2
    test_ebm_corr_df.s_f.to_numpy() # 6 -1
]

In [11]:
# Write data for error analysis

error_analysis_ebm_p = pd.DataFrame({'tokens' : test_ebm_data.tokens,
                                'participant' : test_ebm_data.p,
                                'participant_fine' : test_ebm_data.p_f }, 
                                columns=['tokens','participant', 'participant_fine'])

#error_analysis_ebm_p.to_csv (r'/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/error_analysis/test_ebmgold_p', index = None, header=True) 

In [12]:
# Write data for error analysis

error_analysis_ebmcorr_p = pd.DataFrame({'tokens' : test_ebm_corr_df.tokens,
                                'participant' : test_ebm_corr_df.p,
                                'participant_fine' : test_ebm_corr_df.p_f }, 
                                columns=['tokens','participant', 'participant_fine'])

#error_analysis_ebmcorr_p.to_csv (r'/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/error_analysis/test_ebmgoldcorr_p', index = None, header=True) 

In [13]:
def df_to_list(data_column):
    return [ word for index, value in data_column.items() for word in ast.literal_eval(value) ]

In [14]:
def df_to_array(data_column):
    return np.array( [ word for index, value in data_column.items() for word in ast.literal_eval(value) ] )

In [15]:
def dict_to_array(label_column):
    return np.array( [ labelModel_mapper_LF[int(lab)] for index, value in label_column.items() for k, lab in ast.literal_eval(value).items() ] )

In [16]:
def get_lfs(indir):
    
    pathlist = Path(indir).glob('**/*.tsv')

    tokens = ''

    lfs = dict()
    lfs_lm = dict()

    for counter, file in enumerate(pathlist):
        
        #print(str(file))
        if '/O/' in str(file):

            k = str( file ).split('/v4/')[-1].replace('.tsv', '').replace('/', '_')
            mypath = Path(file)
            if mypath.stat().st_size != 0:
                data = pd.read_csv(file, sep='\t', header=0)

                data_tokens = data.tokens
                if len(tokens) < 5:
                    tokens = df_to_array(data_tokens)

                data_labels = data.labels
                #print(len(data_labels))
                labels = dict_to_array(data_labels)
                #print(len(labels))
                if len(labels) != len(tokens):
                    print(k, len(labels) , len(tokens) )
                #assert len(labels) == len(tokens)
                lfs[k] = labels


    print( 'Total number of tokens in validation set: ', len(tokens) )
    print( 'Total number of LFs in the dictionary', len(lfs) )
    
    return lfs

In [17]:
indir = '/mnt/nas2/results/Results/systematicReview/distant_pico/training_ebm_candidate_generation/v4'
train_ebm_lfs = get_lfs(indir)

Total number of tokens in validation set:  1303169
Total number of LFs in the dictionary 277


In [18]:
indir_test_ebm_corr = '/mnt/nas2/results/Results/systematicReview/distant_pico/test_ebm_anjani_candidate_generation/v4'
test_ebm_corr_lfs = get_lfs(indir_test_ebm_corr)

Total number of tokens in validation set:  52582
Total number of LFs in the dictionary 277


In [19]:
indir_test_ebm = '/mnt/nas2/results/Results/systematicReview/distant_pico/test_ebm_candidate_generation/v4'
test_ebm_lfs = get_lfs(indir_test_ebm)

Total number of tokens in validation set:  51784
Total number of LFs in the dictionary 277


In [20]:
# Remove the annotations where there are no positive labels
print('Dropping no positive label LFs')

def drop_nopositive(lfs_d):
    
    dropped_no_positives = dict()

    for k, v in lfs_d.items():

        if len(set(v)) < 3:
            if 1 in list(set(v)):
                dropped_no_positives[k] = v
        else:
            dropped_no_positives[k] = v
            
    return dropped_no_positives

#lfs_dropped = drop_nopositive(lfs)
#print('Number of LFs: ', len(lfs_dropped))
#lfs_ebm_corr_dropped = drop_nopositive(test_ebm_corr_lfs)
#print('Number of LFs: ', len(lfs_ebm_corr_dropped))

Dropping no positive label LFs


In [21]:
def lf_levels(umls_d:dict, pattern:str, picos:str):

    umls_level = dict()

    for key, value in umls_d.items():   # iter on both keys and values
        search_pattern = pattern + picos
        if key.startswith(search_pattern):
            k = str(key).split('_')[-1]
            umls_level[ k ] = value

    return umls_level


# Level 1: UMLS
umls_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

# ------------------------------------------------------------------------

umls_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

# ------------------------------------------------------------------------

umls_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------


# Level 2: non UMLS
nonumls_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

# ------------------------------------------------------------------------


nonumls_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

# ------------------------------------------------------------------------

nonumls_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------

# Level 3: DS
ds_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]


# ------------------------------------------------------------------------

ds_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

# ------------------------------------------------------------------------

ds_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------


# Level 4: dictionary, rules, heuristics
heur_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------

heur_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------

heur_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['heuristics_direct_'])
]


heur_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------

heur_s = [
    lf_levels(train_ebm_lfs, name, 'S') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_s_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'S') 
    for i, name in enumerate(['heuristics_direct_'])
]


heur_s_testebm = [
    lf_levels(test_ebm_lfs, name, 'S') 
    for i, name in enumerate(['heuristics_direct_'])
]

# ------------------------------------------------------------------------
# ------------------------------------------------------------------------


dict_p = [
    lf_levels(train_ebm_lfs, name, 'P') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_p_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'P') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_p_testebm = [
    lf_levels(test_ebm_lfs, name, 'P') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

# ------------------------------------------------------------------------

dict_i = [
    lf_levels(train_ebm_lfs, name, 'I') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_i_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'I') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_i_testebm = [
    lf_levels(test_ebm_lfs, name, 'I') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

# ------------------------------------------------------------------------

dict_o = [
    lf_levels(train_ebm_lfs, name, 'O') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_o_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'O') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_o_testebm = [
    lf_levels(test_ebm_lfs, name, 'O') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

# ------------------------------------------------------------------------

dict_s = [
    lf_levels(train_ebm_lfs, name, 'S') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_s_testcorrected = [
    lf_levels(test_ebm_corr_lfs, name, 'S') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_s_testebm = [
    lf_levels(test_ebm_lfs, name, 'S') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

In [22]:
# Participant data

train_candidates = [umls_p[1], nonumls_p[1], ds_p[1], dict_p[1], heur_p[0]]
test_ebm_corr_candidates = [umls_p_testcorrected[1], nonumls_p_testcorrected[1], ds_p_testcorrected[1], dict_p_testcorrected[1], heur_p_testcorrected[0]]
test_ebm_candidates = [umls_p_testebm[1], nonumls_p_testebm[1], ds_p_testebm[1], dict_p_testebm[1], heur_p_testebm[0]]

In [23]:
# Intervention data

train_i_candidates = [umls_i[1], nonumls_i[1], ds_i[1], dict_i[1], heur_i[0]]
test_i_ebm_corr_candidates = [umls_i_testcorrected[1], nonumls_i_testcorrected[1], ds_i_testcorrected[1], dict_i_testcorrected[1], heur_i_testcorrected[0]]
test_i_ebm_candidates = [umls_i_testebm[1], nonumls_i_testebm[1], ds_i_testebm[1], dict_i_testebm[1], heur_i_testebm[0]]

In [24]:
# Outcome data

train_o_candidates = [umls_o[1], nonumls_o[1], ds_o[1], dict_o[1], heur_o[0]]
test_o_ebm_corr_candidates = [umls_o_testcorrected[1], nonumls_o_testcorrected[1], ds_o_testcorrected[1], dict_o_testcorrected[1], heur_o_testcorrected[0]]
test_o_ebm_candidates = [umls_o_testebm[1], nonumls_o_testebm[1], ds_o_testebm[1], dict_o_testebm[1], heur_o_testebm[0]]

In [25]:
# Study Type data

train_s_candidates = [dict_s[1], heur_s[0]]
test_s_ebm_corr_candidates = [dict_s_testcorrected[1], heur_s_testcorrected[0]]
test_s_ebm_candidates = [dict_s_testebm[1], heur_s_testebm[0]]

In [26]:
def getLen(lf):
    for l in lf:
        for k,v in l.items():
            print(len(v))
            
#getLen(umls_i) 
#getLen(nonumls_i) 

In [27]:
# Fetch UMLS ranks

sum_lf_p = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/summaries/lf_p_summary_tuipio3_train.csv'
sum_lf_i = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/summaries/lf_i_summary_tuipio3_train.csv'
sum_lf_o = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/summaries/lf_o_summary_tuipio3_train.csv'


def fetchRank(sum_lf_d, pattern, picos: str, drop_nopos: bool):
    
    drop_nopos_keys = []
    for k,v in train_ebm_lfs.items():
        query = '_'+picos+'_'
        if query in str(k):
            key = str(k).split('_lf_')[-1]
            drop_nopos_keys.append(key)
    
    ranked_umls_coverage = dict()    
    umls_coverage_ = dict()
    
    data=pd.read_csv(sum_lf_d, sep='\t')
    
    for index, row in data.iterrows():
        if row[0].startswith(pattern):
            umls_coverage_[row[0]] = row[3]
    
    umls_coverage_sorted = sorted(umls_coverage_.items(), key=lambda x: x[1], reverse=True)
    
    for i in umls_coverage_sorted:
        k = str(i[0]).split('_')[-1]
        if drop_nopos == False:
            ranked_umls_coverage[k] = i[1]
        else:
            if k in drop_nopos_keys:
                ranked_umls_coverage[k] = i[1]

    return ranked_umls_coverage


# fuzzy UMLS
ranksorted_p_umls_fuzzy = fetchRank(sum_lf_p, 'UMLS_fuzzy_', picos='P', drop_nopos = False)
ranksorted_i_umls_fuzzy = fetchRank(sum_lf_i, 'UMLS_fuzzy_', picos='I', drop_nopos = False)
ranksorted_o_umls_fuzzy = fetchRank(sum_lf_o, 'UMLS_fuzzy_', picos='O', drop_nopos = False)


# direct UMLS
ranksorted_p_umls_direct = fetchRank(sum_lf_p, 'UMLS_direct_', picos='P', drop_nopos = False)
ranksorted_i_umls_direct = fetchRank(sum_lf_i, 'UMLS_direct_', picos='I', drop_nopos = False)
ranksorted_o_umls_direct = fetchRank(sum_lf_o, 'UMLS_direct_', picos='O', drop_nopos = False)

proper_coverage_p = '/mnt/nas2/results/Results/systematicReview/distant_pico/coverage_results/participant_UMLS_v3_coverage.json'

with open(proper_coverage_p, 'r') as rf:
    data = json.load(rf)
 
    for k, v in data.items():
        if k in ranksorted_p_umls_fuzzy:
            ranksorted_p_umls_fuzzy[k] = data[k]
            
ranksorted_coverage_p_umls_fuzzy = sorted(ranksorted_p_umls_fuzzy.items(), key=lambda x: x[1], reverse=True)
ranksorted_coverage_p_umls_fuzzy = dict(ranksorted_coverage_p_umls_fuzzy)


proper_coverage_i = '/mnt/nas2/results/Results/systematicReview/distant_pico/coverage_results/intervention_UMLS_v3_coverage.json'

with open(proper_coverage_i, 'r') as rf:
    data = json.load(rf)
 
    for k, v in data.items():
        if k in ranksorted_i_umls_fuzzy:
            ranksorted_i_umls_fuzzy[k] = data[k]
            
ranksorted_coverage_i_umls_fuzzy = sorted(ranksorted_i_umls_fuzzy.items(), key=lambda x: x[1], reverse=True)
ranksorted_coverage_i_umls_fuzzy = dict(ranksorted_coverage_i_umls_fuzzy)

In [28]:
# Partition LF's

def partitionLFs(umls_d):
    
    keys = list(umls_d.keys())

    partitioned_lfs = [ ]
    
    for i in range( 0, len(keys) ):

        if i == 0 or i == len(keys):
            if i == 0:
                partitioned_lfs.append( [keys] )
            if i ==len(keys):
                temp3 = list2Nested(keys, 1)
                partitioned_lfs.append( temp3 )
        else:
            temp1, temp2 = keys[:i] , keys[i:]
            temp3 = list2Nested( keys[:i], 1)
            temp3.append( keys[i:] )
            partitioned_lfs.append( temp3 )
    
    return partitioned_lfs


partitioned_p_umls_fuzzy = partitionLFs(ranksorted_coverage_p_umls_fuzzy)
partitioned_i_umls_fuzzy = partitionLFs(ranksorted_coverage_i_umls_fuzzy)
partitioned_o_umls_fuzzy = partitionLFs(ranksorted_o_umls_fuzzy)

partitioned_p_umls_direct = partitionLFs(ranksorted_p_umls_direct)
partitioned_i_umls_direct = partitionLFs(ranksorted_i_umls_direct)
partitioned_o_umls_direct = partitionLFs(ranksorted_o_umls_direct)

In [29]:
#exp_level = ['UMLS', 'UMLS_Ontology', 'UMLS_Ontology_Rules']
#exp_level = ['UMLS_Ontology_Rules', 'UMLS_Ontology', 'UMLS']
exp_level = ['UMLS_Ontology', 'UMLS']

In [30]:
param_grid = {
    'lr': [0.001, 0.0001],
    'l2': [0.001, 0.0001],
    'n_epochs': [50, 100, 200, 600, 700, 1000, 2000],
    'prec_init': [0.6, 0.7, 0.8, 0.9],
    'optimizer': ["adamax", "adam", "sgd"],
    'lr_scheduler': ['constant'],
}

In [31]:
def sample_param_grid(param_grid, seed):
    """ Sample parameter grid
    :param param_grid:
    :param seed:
    :return:
    """
    rstate = np.random.get_state()
    np.random.seed(seed)
    params = list(product(*[param_grid[name] for name in param_grid]))
    np.random.shuffle(params)
    np.random.set_state(rstate)
    return params

In [32]:
def compare(s, t):
    return sorted(s) == sorted(t)

def getLFs(partition:list, umls_d:dict, seed_len:int):

    all_lfs_combined = []
    
    for lf in partition: # for each lf in a partition
        
        combine_here = [0] * seed_len

        for sab in lf:
            new_a = list(umls_d[sab])
            old_a = combine_here
            temp_a = []
            
            for o_a, n_a in zip(old_a, new_a):
                               
                if compare([o_a, n_a] ,[-1, 1]) == True:
                    replace_a = max( o_a, n_a )
                    temp_a.append( replace_a )
                    
                elif compare([o_a, n_a] ,[0, 1]) == True:
                    replace_a = max( o_a, n_a )
                    temp_a.append( replace_a )
                    
                elif compare([o_a, n_a] ,[-1, 0]) == True:
                    replace_a = min( o_a, n_a )
                    temp_a.append( replace_a )
                else:
                    temp_a.append( o_a )

            combine_here = temp_a

        all_lfs_combined.append( combine_here )

    return all_lfs_combined

In [33]:
def grid_search(model_class,
                model_class_init,
                param_grid,
                train=None,
                dev=None,
                other_train=None,
                n_model_search=5,
                val_metric='f1_macro',
                seed=0,
                checkpoint_gt_mv=False,
                tag_fmt_ckpnt='IO'):
    
    
    """Simple grid search helper function
    Parameters
    ----------
    model_class
    model_class_init
    param_grid
    train
    dev
    n_model_search
    val_metric
    seed

    Returns
    -------
    """
    
    L_train, Y_train = train
    L_dev, Y_dev = dev

    # sample configs
    params = sample_param_grid(param_grid, seed)[:n_model_search]

    defaults = {'seed': seed}
    best_score, best_config = 0.0, None
    print(f"Grid search over {len(params)} configs")

    for i, config in enumerate(params):
        print(f'[{i}] Label Model')
        config = dict(zip(param_grid.keys(), config))
        config.update({param: value for param, value in defaults.items() if param not in config})

        model = model_class(**model_class_init)
        model.fit(L_train, Y_dev, **config)
        
        y_pred = model.predict(L_dev)
        
        if tag_fmt_ckpnt == 'IO':
            y_gold = np.array([0 if y == 0 else 1 for y in Y_dev])
        else:
            y_gold = Y_dev
            
            
        if -1 in y_pred:
            print("Label model predicted -1 (TODO: this happens inconsistently)")
            continue
            
        # use internal label model scorer to score the prediction
        metrics = model.score(L=L_dev,
                              Y=y_gold,
                              metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                              tie_break_policy='random')
        
    
        # compare learned model against MV on same labeled dev set
        # skip if LM less than MV
        if checkpoint_gt_mv:
            mv_metrics = model.score(L=L_dev,
                                  Y=y_gold,
                                  metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                                  tie_break_policy='random')

            if metrics[val_metric] < mv_metrics[val_metric]:
                continue
                
        if not best_score or metrics[val_metric] > best_score[val_metric]:
            print(config)
            best_score = metrics
            best_config = config
            
            # print training set score if we have labeled data
            if np.any(Y_train):
                y_pred = model.predict(L_train)

                if tag_fmt_ckpnt == 'IO':
                    y_gold = np.array([0 if y == 0 else 1 for y in Y_train])
                else:
                    y_gold = Y_train

                metrics = model.score(L=L_train,
                                      Y=y_gold,
                                      metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                                      tie_break_policy='random')

                print('[TRAIN] {}'.format(' | '.join([f'{m}: {v * 100:2.2f}' for m, v in metrics.items()])))

            print('[DEV]   {}'.format(' | '.join([f'{m}: {v * 100:2.2f}' for m, v in best_score.items()])))
            print('-' * 88)
            
            
    # retrain best model
    print('BEST')
    print(best_config)
    model = model_class(**model_class_init)
    
    
    model.fit(L_train, Y_dev, **best_config)
    return model, best_config, best_score

In [34]:
def predict(part, cands, best_model, gt_labels, exp_l):
    
    if exp_l == 'UMLS':
    
        combined_lf = getLFs( part, cands[0], len(gt_labels) ) # level 1 UMLS
        assert len(part) == len(combined_lf)
        print( 'Total number of UMLS partitions: ', len(part) )
        
    elif exp_l == 'UMLS_Ontology':
    
        combined_lf = getLFs( part, cands[0], len(gt_labels) ) # level 1 UMLS
        assert len(part) == len(combined_lf)
        print( 'Total number of UMLS partitions: ', len(part) )
        combined_lf.extend( list(cands[1].values()) ) # level 2 non-UMLS
        
    elif exp_l == 'UMLS_Ontology_Rules':

        combined_lf = getLFs( part, cands[0], len(gt_labels) ) # level 1 UMLS
        assert len(part) == len(combined_lf)
        print( 'Total number of UMLS partitions: ', len(part) )
        combined_lf.extend( list(cands[1].values()) ) # level 2 non-UMLS
        combined_lf.extend( list(cands[2].values()) ) # level 3 DS - Heur
        combined_lf.extend( list(cands[3].values()) ) # level 4 dict - Heur 
        combined_lf.extend( list(cands[4].values()) ) # level 4 ReGeX, Abb - Heur


    L = np.array( combined_lf )
    L = np.transpose(L)
    
    predictions_probablities = best_model.predict_proba(L)
    predictions = best_model.predict(L)
    groundtruth = np.array(gt_labels) 
    groundtruth_ = [1 if x != 0 else x for x in gt_labels] # XXX if "test_ebm_correct"
    groundtruth = np.array(groundtruth_)

    cr = classification_report( groundtruth, predictions, digits=4 )
    print( cr )
    
    return predictions_probablities

In [35]:
def train(partitioned_d_umls, train_cands, test_cands, test_corr_cands, Y_d, picos, paramgrid, mode):
  
    gold_labels = ''
    gold_labels_fine = ''
    
    
    model_class_init = {
        'cardinality': 2, 
        'verbose': True
    }

    num_hyperparams = functools.reduce(lambda x,y:x*y, [len(x) for x in param_grid.values()])
    print("Hyperparamater Search Space:", num_hyperparams)
    n_model_search = 50
    

    '''#########################################################################
    # Choosing the number of LF's from UMLS all
    #########################################################################'''
    
    for l in exp_level:
        print( 'Executing the experiment level: ', l )
        
        best_f1_macro = 0.0
        best_overall_model = ''
        best_overall_config = ''
        best_overall_partition = 0
        overall_L = ''
    
        for i, partition in enumerate(partitioned_d_umls):
        
            #if len(partition) == 2:
            
            if l == 'UMLS' and i < 2:
                continue

            if l == 'UMLS':

                combined_lf = getLFs(partition, train_cands[0], len( data_df.tokens ))
                #combined_lf = combined_lf[2:]  # i>2 (as LM should have at least 3 labeling functions)
                assert len(partition) == len(combined_lf)
                print( 'Total number of UMLS partitions: ', len(partition) )
                
            if l == 'UMLS_Ontology':

                combined_lf = getLFs(partition, train_cands[0], len( data_df.tokens )) # level 1 UMLS
                assert len(partition) == len(combined_lf)
                print( 'Total number of UMLS partitions: ', len(partition) )
                combined_lf.extend( list(train_cands[1].values()) ) # level 2 non-UMLS
            
            if l == 'UMLS_Ontology_Rules':

                combined_lf = getLFs(partition, train_cands[0], len( data_df.tokens ))
                assert len(partition) == len(combined_lf)
                print( 'Total number of UMLS partitions: ', len(partition) )
                combined_lf.extend( list(train_cands[1].values()) ) # level 2 non-UMLS
                combined_lf.extend( list(train_cands[2].values()) ) # level 3 DS - Heur
                combined_lf.extend( list(train_cands[3].values()) ) # level 4 dict - Heur 
                combined_lf.extend( list(train_cands[4].values()) ) # level 4 ReGeX, Abb - Heur


            L = np.array( combined_lf )
            L = np.transpose(L)
            L_train, L_val = train_test_split(L, test_size=0.20, shuffle=False)
            Y_train, Y_val = train_test_split( np.array(Y_d[0]), test_size=0.20, shuffle=False)
            Y_train_fine, Y_val_fine = train_test_split( np.array(Y_d[1]), test_size=0.20, shuffle=False)

            # convert the fine labels to 0 and 1
            Y_train_fine = [1 if x != 0 else x for x in Y_train_fine]
            Y_val_fine = [1 if x != 0 else x for x in Y_val_fine]

            #print( Y_val_fine )


            #Y_train = Y_d[0]
            #Y_val = Y_d[1]
            Y = np.concatenate([Y_train, Y_val])
            Y_fine = np.concatenate([Y_train_fine, Y_val_fine])

            best_model, best_config, best_score = grid_search(LMsnorkel, 
                                                   model_class_init, 
                                                   paramgrid,
                                                   train = (L_train, Y_train_fine),
                                                   dev = (L_val, Y_val_fine),
                                                   n_model_search=n_model_search, 
                                                   val_metric='f1_macro', 
                                                   seed=seed_i,
                                                   tag_fmt_ckpnt='IO')


            if best_score['f1_macro'] > best_f1_macro:
                best_f1_macro = best_score['f1_macro']
                best_overall_model = best_model
                best_overall_config = best_config
                best_overall_partition = i
                overall_L = L
                gold_labels = Y_d[0]
                gold_labels_fine = Y_d[1]


            print('Best overall macro F1 score: ', best_f1_macro)
            print('Best overall configuration: ', best_overall_config)


        print('Save the best overall model, configuration and partition for this experiment level')
        # Save your model or results
        save_dir = f'/mnt/nas2/results/Results/systematicReview/distant_pico/models/LabelModels/{picos}/v4/{l}/{seed_i}'
        filename = 'stpartition_' + str(best_overall_partition+1) + '_epoch_' + str(best_config['n_epochs'])
        joblib.dump(best_overall_model, f'{save_dir}/{filename}.pkl') 
        joblib.dump(best_overall_config, f'{save_dir}/{filename}.json')

        #load your model for further usage
        loaded_best_model = joblib.load(f'{save_dir}/{filename}.pkl')

        # Initialize token predictions here
        token_predictions = dict()

        # Predict on the training set
        for i, partition in enumerate(partitioned_d_umls):

            if i == best_overall_partition:

                # Predict on the test ebm correct set
                test_corr_probas = predict(partition, test_corr_cands, loaded_best_model, Y_d[-2], l) # test ebm correct   

                # Predict on the test ebm set
                test_ebm_probas = predict(partition, test_cands,loaded_best_model, Y_d[-4], l) # test ebm
                test_ebm_probas_coarse = predict(partition, test_cands,loaded_best_model, Y_d[-5], l) # test ebm coarse grained

                # Predict on the training set
                train_probas = predict(partition, train_cands,loaded_best_model, Y_d[-6], l) # train 

                # Write training predictions to file
                # tokens	pos	offsets	labels	true_labels
                train_probas = [list(tp) for tp in train_probas]
                train_probas_series = pd.Series(list(train_probas))
                data_df['labels'] = train_probas_series.values
                
                # Write predictions on the training data to the file
                write_df = data_df.groupby(['series'])[['series', 'tokens', 'pos', 'offsets', 'labels', str(picos), str(picos)+'_f']].agg(list)
                write_file_path = f'/mnt/nas2/results/Results/systematicReview/distant_pico/predictions/LabelModels/{picos}/v4/{l}/{seed_i}/{filename}_bestmodel_train.tsv'
                write_df.to_csv (write_file_path, index = None, sep = '\t', header=True) 

                # Write predictions for the ebm and ebm corrected files
                #token_predictions = dict()
                #for counter, (T, P, Y_d_i, Y_d_i_fine) in enumerate(zip(tokens, predictions, gold_labels, gold_labels_fine)):
                #    token_predictions[counter] = [T, list(P), list(P).index(max(list(P))), str(Y_d_i), str(Y_d_i_fine)]

        #with open(f'{save_dir}/{filename}_results.json', 'w+') as fp:
        #    json.dump(token_predictions, fp)
            
    #return token_predictions

In [56]:
# Seed 0 (UMLS_Ontology_Rules)
predicted_o = train(partitioned_o_umls_fuzzy, train_o_candidates, test_o_ebm_candidates, test_o_ebm_corr_candidates, Y_o, 'o', paramgrid = param_grid, mode = 'pred')

Hyperparamater Search Space: 336
Executing the experiment level:  UMLS_Ontology_Rules
Total number of UMLS partitions:  5
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 71.01 | precision: 17.23 | recall: 56.80 | f1: 26.44 | f1_macro: 54.19
[DEV]   accuracy: 71.04 | precision: 17.78 | recall: 58.44 | f1: 27.27 | f1_macro: 54.59
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 73.22 | precision: 17.61 | recall: 52.19 | f1: 26.34 | f1_macro: 54.99
[DEV]   accuracy: 73.24 | precision: 17.81 | recall: 52.05 | f1: 26.54 | f1_macro: 55.09
----------------------------------------------------------------------------------------
[3] Label Model
[4] L

KeyboardInterrupt: 

In [ ]:
# Seed 0
predicted_o = train(partitioned_o_umls_fuzzy, train_o_candidates, test_o_ebm_candidates, test_o_ebm_corr_candidates, Y_o, 'o', paramgrid = param_grid, mode = 'pred')

Hyperparamater Search Space: 336
Executing the experiment level:  UMLS_Ontology
Total number of UMLS partitions:  1
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 88.63 | precision: 23.35 | recall: 10.50 | f1: 14.49 | f1_macro: 54.20
[DEV]   accuracy: 88.56 | precision: 23.40 | recall: 10.18 | f1: 14.19 | f1_macro: 54.03
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 73.90 | precision: 17.19 | recall: 48.33 | f1: 25.36 | f1_macro: 54.77
[DEV]   accuracy: 73.86 | precision: 17.55 | recall: 49.08 | f1: 25.86 | f1_macro: 55.00
----------------------------------------------------------------------------------------
[3] Label Model
[4] Label M

[TRAIN] accuracy: 77.95 | precision: 18.07 | recall: 39.69 | f1: 24.83 | f1_macro: 55.96
[DEV]   accuracy: 77.98 | precision: 18.75 | recall: 41.14 | f1: 25.76 | f1_macro: 56.42
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
[4] Label Model
[5] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 84.03 | precision: 22.32 | recall: 29.86 | f1: 25.54 | f1_macro: 58.30
[DEV]   accuracy: 84.01 | precision: 22.75 | recall: 30.13 | f1: 25.93 | f1_macro: 58.48
----------------------------------------------------------------------------------------
[6] Label Model
[7] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 86.41 | precision: 25.39 | recall: 24.80 | f1: 25.09 | f1_macro: 58.81
[DEV]   accuracy:

Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 79.44 | precision: 19.37 | recall: 39.22 | f1: 25.93 | f1_macro: 57.00
[DEV]   accuracy: 79.47 | precision: 19.99 | recall: 40.34 | f1: 26.74 | f1_macro: 57.40
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
[3] Label Model
[4] Label Model
[5] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 82.18 | precision: 21.04 | recall: 34.21 | f1: 26.05 | f1_macro: 57.96
[DEV]   accuracy: 82.19 | precision: 21.56 | recall: 34.78 | f1: 26.62 | f1_macro: 58.24
----------------------------------------------------------------------------------------
[6] Label Model
[7] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.

{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 83.83 | precision: 22.43 | recall: 31.00 | f1: 26.03 | f1_macro: 58.48
[DEV]   accuracy: 83.74 | precision: 22.60 | recall: 30.99 | f1: 26.14 | f1_macro: 58.50
----------------------------------------------------------------------------------------
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
Best overall macro F1 score:  0.5923475986026187
Best overall configuration:  {'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
Total numb

[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 83.49 | precision: 21.79 | recall: 30.88 | f1: 25.55 | f1_macro: 58.14
[DEV]   accuracy: 83.46 | precision: 22.17 | recall: 31.12 | f1: 25.90 | f1_macro: 58.29
----------------------------------------------------------------------------------------
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 83.44 | precision: 21.79 | recall: 31.08 | f1: 25.62 | f1_macro: 58.15
[DEV]   accuracy: 83.41 | precision: 22.19 | recall: 31.37 | f1: 26.00 | f1_macro: 58.33
------------------------------------------------------

...

[TRAIN] accuracy: 83.94 | precision: 21.28 | recall: 27.79 | f1: 24.10 | f1_macro: 57.56
[DEV]   accuracy: 83.95 | precision: 21.94 | recall: 28.47 | f1: 24.78 | f1_macro: 57.90
----------------------------------------------------------------------------------------
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
Best overall macro F1 score:  0.5897347446248995
Best overall configuration:  {'lr': 0.001, 'l2': 0.0001, 'n_epochs': 100, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 0}
Total number of UMLS partitions:  112
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec

[TRAIN] accuracy: 85.51 | precision: 21.62 | recall: 22.08 | f1: 21.85 | f1_macro: 56.93
[DEV]   accuracy: 85.49 | precision: 22.05 | recall: 22.20 | f1: 22.13 | f1_macro: 57.06
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 81.24 | precision: 19.91 | recall: 34.58 | f1: 25.27 | f1_macro: 57.27
[DEV]   accuracy: 81.09 | precision: 20.10 | recall: 34.83 | f1: 25.49 | f1_macro: 57.33
----------------------------------------------------------------------------------------
[3] Label Model
[4] Label Model
[5] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 83.43 | precision: 20.54 | recall: 28.08 | f1: 23.72 | f1_macro: 57.22
[DEV]   accuracy: 83.38 | precision: 21.02 | recal

{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.7, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 82.31 | precision: 20.18 | recall: 31.40 | f1: 24.57 | f1_macro: 57.28
[DEV]   accuracy: 82.20 | precision: 20.57 | recall: 32.03 | f1: 25.05 | f1_macro: 57.48
----------------------------------------------------------------------------------------
[7] Label Model
{'lr': 0.0001, 'l2': 0.0001, 'n_epochs': 1000, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 84.62 | precision: 21.74 | recall: 26.01 | f1: 23.68 | f1_macro: 57.57
[DEV]   accuracy: 84.64 | precision: 22.37 | recall: 26.49 | f1: 24.25 | f1_macro: 57.85
----------------------------------------------------------------------------------------
[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model


[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 83.94 | precision: 21.28 | recall: 27.78 | f1: 24.10 | f1_macro: 57.56
[DEV]   accuracy: 83.95 | precision: 21.94 | recall: 28.47 | f1: 24.78 | f1_macro: 57.90
----------------------------------------------------------------------------------------
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'sg

[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
Label model predicted -1 (TODO: this happens inconsistently)
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.7, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 0}
Best overall macro F1 score:  0.5897347446248995
Best overall configuration:  {'lr': 0.001, 'l2': 0.0001, 'n_epochs': 100, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 0}
Total number of UMLS partitions:  121
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 0}
[TRAIN] accuracy: 85.51 | precision: 21.62 | recall: 22.08 | f1: 21.85 | f1_macro: 56.93
[DEV]   accuracy: 85.49 | precision: 22.05 | recall: 22.20 

In [ ]:
exp_level = ['UMLS_Ontology_Rules', 'UMLS_Ontology', 'UMLS']

In [ ]:
# Seed 1 # actually set the seed to 1
predicted_o = train(partitioned_o_umls_fuzzy, train_o_candidates, test_o_ebm_candidates, test_o_ebm_corr_candidates, Y_o, 'o', paramgrid = param_grid, mode = 'pred')

In [ ]:
# Seed 42

seed_i = 42

predicted_o = train(partitioned_o_umls_fuzzy, train_o_candidates, test_o_ebm_candidates, test_o_ebm_corr_candidates, Y_o, 'o', paramgrid = param_grid, mode = 'pred')